Library Insatallation

In [ ]:
!pip install kaggle
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 175.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
%uv pip install -U unsloth unsloth-zoo transformers peft accelerate --quiet
%uv pip install trl==0.23.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# **Load Data And Checkpoint**

In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d fatmaalzhraahmed/shifaa-arabic-medical-q-and-a

Dataset URL: https://www.kaggle.com/datasets/fatmaalzhraahmed/shifaa-arabic-medical-q-and-a
License(s): apache-2.0
100%|█████████████████████████████████████▊| 78.0M/78.4M [00:03<00:00, 43.1MB/s]
100%|██████████████████████████████████████| 78.4M/78.4M [00:03<00:00, 23.6MB/s]


In [ ]:
!unzip shifaa-arabic-medical-q-and-a.zip

Archive:  shifaa-arabic-medical-q-and-a.zip
  inflating: Shifaa Arabic Medical Q and A.csv  


In [ ]:
!modal volume ls my-saved-models /checkpoint-1000

             Directory listing of '/checkpoint-1000' in 'my-saved-models'             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Filename                                  ┃ Type ┃ Created/Modified     ┃ Size     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ checkpoint-1000/README.md                 │ file │ 2025-11-26 22:55 UTC │ 5.1 KiB  │
│ checkpoint-1000/adapter_config.json       │ file │ 2025-11-26 22:55 UTC │ 1.2 KiB  │
│ checkpoint-1000/adapter_model.safetensors │ file │ 2025-11-26 22:55 UTC │ 2.1 GiB  │
│ checkpoint-1000/added_tokens.json         │ file │ 2025-11-26 22:55 UTC │ 605 B    │
│ checkpoint-1000/chat_template.jinja       │ file │ 2025-11-26 22:55 UTC │ 2.4 KiB  │
│ checkpoint-1000/merges.txt                │ file │ 2025-11-26 22:55 UTC │ 1.6 MiB  │
│ checkpoint-1000/optimizer.pt              │ file │ 2025-11-26 22:55 UTC │ 2.1 GiB  │
│ checkpoint-1000/rng_state.pth            

In [ ]:
!mkdir checkpoint-1000

In [ ]:
!modal volume get my-saved-models /checkpoint-1000 checkpoint-1000

⠋ Downloading file(s) to local...
⠸ Downloading file(s) to local...
⠦ Downloading file(s) to local...
⠏ Downloading file(s) to local...
                checkpoint-1000/README.md ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/5.3 kB  • ? • -:--:--
      checkpoint-1000/adapter_config.json ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/1.2 kB  • ? • -:--:--
checkpoint-1000/adapter_model.safetensors ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/2.3 GB  • ? • -:--:--
        checkpoint-1000/added_tokens.json ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0/605 bytes • ? • -:--:--
      checkpoint-1000/chat_template.jinja ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/2.5 kB  • ? • -:--:--
               checkpoint-1000/merges.txt ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/1.7 MB  • ? • -:--:--
             checkpoint-1000/optimizer.pt ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/2.3 GB  • ? • -:--:--
            checkpoint-1000/rng_state.pth ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#Train Model on 10% from dataset model
## Split Data Into Train, Test, Val
### With size :


1.   Train : 80% (**6753**)
2.   Validation: 15% (**1266**)
3.   Test :5%  (**423**)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.model_selection import train_test_split
from datasets import load_dataset,DatasetDict

In [ ]:
df=pd.read_csv("Shifaa Arabic Medical Q and A.csv")
df

,Question,Answer
0,أعاني من ألم فوق وبجانب السرة وكأنه طعن بسكين ...,منى إن الأدوية التي تستعملينها (ميتوتركسات وال...
1,عمري 31 سنة ومتزوجة منذ 8 أشهر ولم يحصل الحمل ...,سارة أتفهم مشاعرك ولهفتك على الحمل يا عزيزتي ف...
2,سؤالي عندي طفل يبلغ مع العمر 6 سنوات المشكلة ه...,قد يتسبب الإمساك المزمن عند الطفل بسبب نقص الس...
3,تعرضت لحادث في المباراة وعملت صورة رنين مغناطي...,حبذا لو ذكرت لنا كم مضى على تمزق الرباط وهل هو...
4,أنا شاب عمري 22 عاما أعاني من القولون العصبي م...,لا يمكن لمن يتناول الحلويات والسكريات والمشروب...
...,...,...
84417,أريد أن أعرف ما هي أسباب اسمرار مفاصل اليد حتى...,سميرة المناطق المعرضة للاحتكاك بالمحيط الخارجي...
84418,أعاني من كثافة الشعر على الساعدين والرجلين بشك...,فإن الاختلاف بين الناس وخلقهم وطبائعهم هو أمر ...
84419,ما هو مرض الزلال فقد قمت بفحص البول وأخبرني ال...,الأخ الفاضل أبو زهور الزلال كلمة تطلق على ظهور...
84420,أنا فريال من الجزائر من فضلكم أريد أن أعرف متى...,فريال وبعد عوضك الله بكل خير وجعل صبرك واحتساب...


In [ ]:
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [ ]:
data_for_model = []
for index, row in df.iterrows():
    data_for_model.append({
        "instruction": row['Question'],
        "output": row['Answer']
    })

In [ ]:
with open('data_read_for_model.jsonl', 'w', encoding='utf-8') as f:
    for i in data_for_model:
        f.write(json.dumps(i, ensure_ascii=False) + "\n")

In [ ]:
full_dataset = load_dataset("json", data_files="data_read_for_model.jsonl", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_test_split = full_dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
validation_test_split = train_test_split['test'].train_test_split(test_size=0.25, seed=42)

In [ ]:
final_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': validation_test_split['train'],
    'test': validation_test_split['test']
})

In [ ]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 6753
    })
    validation: Dataset({
        features: ['instruction', 'output'],
        num_rows: 1266
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 423
    })
})

In [ ]:
train_dataset =  final_dataset['train']
val_dataset = final_dataset['validation']
test_dataset = final_dataset['test']

In [ ]:
train_dataset[6]

{'instruction': 'كسرت قصبة ساقي أثناء ممارسة كرة القدم وتم إسعافي وأثناء الفحص السريري تبين أن هناك كسرا من النوع الكامل وأن العظم غير متزحزح وأن العظم على العظم ولا داعي للعملية ولا يوجد أي إصابة للورك وتم التثبيت بواسطة الجبس لمنتصف الفخذ وبعد مرور 6 أسابيع تم تخفيف الجبس أسفل الركبة بشكل بسيط سؤالي كم الفترة اللازمة لإزالة الجبس كاملا ومتى أستطيع المشي بدون أي عكازات مع العلم أن الركبة غير متضررة نتيجة الجبس وعمري 28 سنة',
 'output': 'عادة الكسر غير المتبدل يلتحم بمدة أسرع من الكسر المتبدل وعادة تتفاوت مدة الالتحام حسب العظم المكسور وعمر المريض وفيما إذا كان المريض عظامه سليمة أو أن لديه مشاكل عظمية كهشاشة وتلين العظام أما بالنسبة لكسرك فأعتقد أنه بعد مرور ستة أسابيع قد قارب على الالتحام التام لذلك يمكنك المشي بمساعدة العكازات لمدة أسبوعين وبعدها تستطيع نزع الجبس والمشي بدون عكازات ونفضل إجراء صورة شعاعية بعد أسبوعين وعرضها على طبيبك المعالج وهو بدوره سوف يأخذ القرار الصحيح بعد قراءة الصورة ونسأل الله لك الشفاء العاجل'}

#Loading Model

In [ ]:
from huggingface_hub import login
token = ""
login(token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,AutoModelForSequenceClassification
from unsloth import FastLanguageModel

/tmp/ipykernel_355/4100157713.py:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
import torch
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-7B-Instruct",
    max_seq_length=1024,
    token=token,
    quantization_config=bnb_config,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.11.4: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
import torch.nn as nn
class MedicalAdapterLayer(nn.Module):

    def __init__(self, hidden_size=3584, bottleneck_size=512):
        super().__init__()
        self.medical_projection = nn.Sequential(
            nn.Linear(hidden_size, bottleneck_size),
            nn.GELU(),
            nn.Dropout(0.05),
            nn.Linear(bottleneck_size, hidden_size),
            nn.LayerNorm(hidden_size)
        )

    def forward(self, hidden_states):
        return hidden_states + 0.1 * self.medical_projection(hidden_states)

In [ ]:
medical_adapter = MedicalAdapterLayer(hidden_size=3584, bottleneck_size=512).to(model.device)

# Wrap the model's forward pass
original_forward = model.forward


In [ ]:
if hasattr(model, 'base_model'):
    base_model = model.base_model.model if hasattr(model.base_model, 'model') else model.base_model
else:
    base_model = model

original_model_forward = base_model.__class__.forward

In [ ]:
def forward_with_medical_adapter(self, input_ids=None, attention_mask=None, **kwargs):
    kwargs['output_hidden_states'] = True
    outputs = original_model_forward(self, input_ids=input_ids, attention_mask=attention_mask, **kwargs)

    if hasattr(outputs, 'hidden_states') and outputs.hidden_states is not None and len(outputs.hidden_states) > 0:
        last_hidden = outputs.hidden_states[-1]

        with torch.cuda.amp.autocast(enabled=False):
            adapted = medical_adapter(last_hidden.float())

        outputs.hidden_states = outputs.hidden_states[:-1] + (adapted,)

    return outputs

In [ ]:
import types

base_model.forward = types.MethodType(forward_with_medical_adapter, base_model)

model.medical_adapter = medical_adapter

In [ ]:
print(f"Medical adapter parameters: {sum(p.numel() for p in medical_adapter.parameters()):,}")

Medical adapter parameters: 3,681,280


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        "embed_tokens",
        "lm_head"
    ],
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
/usr/local/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)
Unsloth 2025.11.4 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [ ]:
for param in model.medical_adapter.parameters():
    param.requires_grad = True

In [ ]:
print("\nLoRA configuration applied successfully!")
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_params += sum(p.numel() for p in model.medical_adapter.parameters())
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters (including medical adapter): {trainable_params:,}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")


LoRA configuration applied successfully!
Trainable parameters (including medical adapter): 1,113,861,120
Total parameters: 6,005,408,256
Trainable %: 18.55%


In [ ]:
qwen_prompt_template = """<|im_start|>system
أنت "شِفاء"، مساعد طبي متخصص وموثوق. مهمتك تقديم إجابات طبية دقيقة وواضحة باللغة العربية. استخدم أسلوب متعاطف ومهني، ونظم إجاباتك بشكل منطقي ومفهوم.<|im_end|>
<|im_start|>user
{}<|im_end|>
<|im_start|>assistant
{}<|im_end|>"""

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = qwen_prompt_template.format(instruction, output)
        texts.append(text)
    return {"text": texts}

In [ ]:
train_dataset_formatted = train_dataset.map(formatting_prompts_func, batched = True,)
val_dataset_formatted = val_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/6753 [00:00<?, ? examples/s]

Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

In [ ]:
train_dataset_formatted['text'][40]

'<|im_start|>system\nأنت "شِفاء"، مساعد طبي متخصص وموثوق. مهمتك تقديم إجابات طبية دقيقة وواضحة باللغة العربية. استخدم أسلوب متعاطف ومهني، ونظم إجاباتك بشكل منطقي ومفهوم.<|im_end|>\n<|im_start|>user\nأنا أعاني من تشنجات وقبل أن أفقد الوعي تأتي لي رعشات في جميع أنحاء جسدي تأتي نحو 5 أو 6 مرات قبل أن أفقد الوعي وأصاب بنوبة صرع ماذا أفعل عندما تأتي لي هذه الأعراض لتفادي الصرع أو لتفادي الإغماء وشكرا<|im_end|>\n<|im_start|>assistant\nأرحب بك في الشبكة الإسلامية وأسأل الله لك العافية والشفاء أيها الابن الفاضل لا بد أن تذهب إلى الطبيب اذهب لطبيب المخ والأعصاب المختص في أمراض التشنجات خاصة الأمراض الصرعية وهذه الحالات الآن تعالج وتعالج بصورة ممتازة جدا الطبيب ربما يجري لك بعض الفحوصات مثل تخطيط الدماغ أو صورة مقطعية مثلا للرأس وبعض فحوصات الدم كذلك للتأكد فقط أرجو ألا تتأخر أبدا اذهب إلى الطبيب وسوف يشرح لك كل شيء وسوف يعطيك العلاج اللازم وأنا أبشرك أنه توجد أدوية كثيرة جدا لعلاج هذه التشنجات هنالك عقار يسمى تجاريا (تجراتول Tegretol) وآخر يسمى (ايبانوتين Epanutin) وثالث يسمى (ديباكين Depakine)

In [ ]:
print(train_dataset_formatted['text'][40])

<|im_start|>system
أنت "شِفاء"، مساعد طبي متخصص وموثوق. مهمتك تقديم إجابات طبية دقيقة وواضحة باللغة العربية. استخدم أسلوب متعاطف ومهني، ونظم إجاباتك بشكل منطقي ومفهوم.<|im_end|>
<|im_start|>user
أنا أعاني من تشنجات وقبل أن أفقد الوعي تأتي لي رعشات في جميع أنحاء جسدي تأتي نحو 5 أو 6 مرات قبل أن أفقد الوعي وأصاب بنوبة صرع ماذا أفعل عندما تأتي لي هذه الأعراض لتفادي الصرع أو لتفادي الإغماء وشكرا<|im_end|>
<|im_start|>assistant
أرحب بك في الشبكة الإسلامية وأسأل الله لك العافية والشفاء أيها الابن الفاضل لا بد أن تذهب إلى الطبيب اذهب لطبيب المخ والأعصاب المختص في أمراض التشنجات خاصة الأمراض الصرعية وهذه الحالات الآن تعالج وتعالج بصورة ممتازة جدا الطبيب ربما يجري لك بعض الفحوصات مثل تخطيط الدماغ أو صورة مقطعية مثلا للرأس وبعض فحوصات الدم كذلك للتأكد فقط أرجو ألا تتأخر أبدا اذهب إلى الطبيب وسوف يشرح لك كل شيء وسوف يعطيك العلاج اللازم وأنا أبشرك أنه توجد أدوية كثيرة جدا لعلاج هذه التشنجات هنالك عقار يسمى تجاريا (تجراتول Tegretol) وآخر يسمى (ايبانوتين Epanutin) وثالث يسمى (ديباكين Depakine) ورابع

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding=False,
    )

train_tokenized = train_dataset_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset_formatted.column_names,
)

val_tokenized = val_dataset_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset_formatted.column_names,
)


Map:   0%|          | 0/6753 [00:00<?, ? examples/s]

Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="qwen_arabic_medical",

    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,

    warmup_steps=10,
    num_train_epochs=3,

    learning_rate=2e-4,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",

    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    gradient_checkpointing=True,
    max_grad_norm=0.3,

    logging_steps=25,
    eval_strategy="steps",
    eval_steps=250,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=2,
    report_to="wandb",
    run_name=f"qwen-medical-lr-{2e-4}-ep-{1}",

    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    seed=3407,

    dataloader_pin_memory=False,
    dataloader_num_workers=0,
)


In [ ]:
import torch
from transformers import Trainer
class MedicalTrainer(Trainer):
    def create_optimizer(self):
        if self.optimizer is None:
            decay_parameters = []
            all_trainable_params = set()

            for n, p in self.model.named_parameters():
                if p.requires_grad:
                    decay_parameters.append(p)
                    all_trainable_params.add(id(p))

            for p in self.model.medical_adapter.parameters():
                if id(p) not in all_trainable_params:
                    decay_parameters.append(p)
                    all_trainable_params.add(id(p))

            optimizer_grouped_parameters = [
                {
                    "params": decay_parameters,
                    "weight_decay": self.args.weight_decay,
                }
            ]

            optimizer_cls, optimizer_kwargs = Trainer.get_optimizer_cls_and_kwargs(self.args)
            self.optimizer = optimizer_cls(optimizer_grouped_parameters, **optimizer_kwargs)

        return self.optimizer


In [ ]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 107.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
trainer = MedicalTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

[accelerate.utils.other|WARNING]Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
import wandb
wandb.login(key="144b62e4cc025d1bd808e82a62182780926b0d5b")
trainer.train(resume_from_checkpoint="/root/qwen_arabic_medical/checkpoint-2000")
#144b62e4cc025d1bd808e82a62182780926b0d5b

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,753 | Num Epochs = 3 | Total steps = 2,535
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 1,113,861,120 of 8,729,477,632 (12.76% trained)
/tmp/ipykernel_355/257082065.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


Step,Training Loss,Validation Loss
